In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
# from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import TimeoutException
# from selenium.common.exceptions import WebDriverException
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup
import time
import os
import csv

In [ ]:
chrome_options = Options()   # chrome 옵션 객체 생성
chrome_options.add_experimental_option("detach", True) # 브라우져 창이 열린 상태로 유지되도록 설정
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
service = Service(executable_path=ChromeDriverManager().install()) # chrome 드라이버를 실행하기 위한 service 객체 생성

In [ ]:
# 카테고리 클릭하기
from selenium.common.exceptions import NoSuchElementException
browser = webdriver.Chrome(service=service,options=chrome_options)
wait = WebDriverWait(browser, 10)
browser.maximize_window()
url = "https://www.oliveyoung.co.kr/store/display/getMCategoryList.do?dispCatNo=100000100020006&fltDispCatNo=&prdSort=01&pageIdx=1&rowsPerPage=24&searchTypeSort=btn_thumb&plusButtonFlag=N&isLoginCnt=0&aShowCnt=0&bShowCnt=0&cShowCnt=0&trackingCd=Cat100000100020006_MID"
browser.get(url)
wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#Contents')))
total_links = []
for i in range(2,6):
    
    selector = f'#Contents > ul.cate_list_box > li:nth-child({i}) > a'
    next_tab = browser.find_element(By.CSS_SELECTOR, selector)
    next_tab.click()

    while True:
        try:
            wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#Container > div.pageing > strong')))
            soup = BeautifulSoup(browser.page_source,'html.parser')
            links = soup.find_all('a', class_=['prd_thumb'])
            links = [_['href'] for _ in links]
            # for a in links:
            #     total_links.append(a)
            total_links.append(links)
            next_num = browser.find_element(By.CSS_SELECTOR, '#Container > div.pageing > strong + a')
            next_num.click()
        except Exception:
            print("페이지네이션 끝")
            break
browser.quit()

In [ ]:
len(total_links)

In [ ]:
product_data = []

for link in total_links:
    for item in link:  
        # selenium을 사용하기 위한
        browser = webdriver.Chrome()
        browser.maximize_window()
        browser.get(item)
        wait = WebDriverWait(browser, 3)
        css_selector = 'div.left_area > div.prd_colorchip_list.zip > button.btn_more_colorchip'

        # 제목과 가격
        title = browser.find_element(By.CSS_SELECTOR,'#Contents > div.prd_detail_box.renew > div.right_area > div > p.prd_name').text
        time.sleep(2)
        price = browser.find_element(By.CSS_SELECTOR,'#Contents > div.prd_detail_box.renew > div.right_area > div > div.price > span > strong').text
        time.sleep(2)

        try:
            element = browser.find_element(By.CSS_SELECTOR, css_selector)
            element.click()
        except NoSuchElementException:
            pass

        n = browser.find_elements(By.CSS_SELECTOR, '.thumb-color')

        for n in range(len(n)):
            data = []
            browser.find_element(By.CSS_SELECTOR, f'#colrParet_{n}').click()
            color_img = browser.find_element(By.CSS_SELECTOR, f'#colrParet_{n} > span > img').get_attribute('src')
            time.sleep(2)
            main_img = browser.find_element(By.CSS_SELECTOR, '#mainImg').get_attribute('src')
            time.sleep(2)
            img_name = browser.find_element(By.CSS_SELECTOR, '#goodstxt').text

            data.append(title)
            data.append(price)
            data.append(color_img)
            data.append(main_img)
            data.append(img_name)
            product_data.append(data)
            
browser.quit()


In [ ]:
product_data

In [ ]:
pro_df = pd.DataFrame(product_data, columns=['title','price','color_img','main_img','color_name'])

In [ ]:
pro_df.to_csv('c:/project/pro_df.csv', sep=',', encoding='utf-8-sig')

In [ ]:
pro_df.title = pro_df.title.str.replace('/','_')

In [ ]:
# 이미지 저장 폴더 생성
import os
titles = pro_df.title.unique()
for title in titles:
    os.mkdir('c:/project/image/'+str(title))

In [ ]:
# 상품 이름 경로에 컬러명으로 이미지 저장

for item in range(len(pro_df.index)):
    title = pro_df.loc[item].title
    color_name = pro_df.loc[item].color_name
    color_img = pro_df.loc[item].color_img
    main_img = pro_df.loc[item].main_img

    urllib.request.urlretrieve(main_img, 'c:/project/image/'+title+'/'+color_name+'(발색비교).jpg')
    urllib.request.urlretrieve(color_img, 'c:/project/image/'+title+'/'+color_name+'(상품색상).jpg')

In [ ]:
pro_df = pro_df.rename(columns={'title': '품명', 'color_name': '색상'})

In [ ]:
pro_df['이미지경로_발색비교']=pro_df.apply(lambda x: x.품명+'/'+x.색상+'(발색비교).jpg', axis=1)
pro_df['이미지경로_상품색상']=pro_df.apply(lambda x: x.품명+'/'+x.색상+'(상품색상).jpg', axis=1)

In [ ]:
pro_df